# Stage Based Calibration

This notebook allows the user to adjust calibration parameters to find sensible starting values before attempting to fit. 

In [1]:
import volteracamera.intrinsics.stage_calibration as sc
from volteracamera.analysis.undistort import Undistort
from volteracamera.analysis.transform import Transform
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
from IPython.display import display

input_file = "/home/rwicks/Voltera/camera/paper_cal_output.csv"

In [2]:
data = np.loadtxt(input_file, delimiter=",")

#filter out bad point (y > 7) for the original datasets. Need to add better filters.
position = []
points_2d = []
for x, y, z, u, v in data:
    if y >=7:
        continue
    position.append(np.array([x, y, z])/1000)
    points_2d.append(np.array([u, v]))

In [3]:
def points_projected (points_3d, points_2d, fx=6325, fy=6325, cx=1640, cy=1232, rx=-0.11327094, ry=-0.0359958, rz=1.55522712, tx=0.001659, ty=-0.00299, tz=0.019462):
    """
    Get residuals of projected 3d points, compared to the imaged points (two points for each points)
    """
    cam_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]) 
    distortion = np.array([0, 0, 0, 0, 0])
    undistort = Undistort(cam_matrix, np.array([0, 0, 0, 0, 0]))
    transform = Transform(rotation=[rx, ry, rz], translation=[tx, ty, tz])
    projected = np.array([ sc.get_projected(point_3d, undistort, transform) for point_3d in points_3d])
    residuals = np.array([proj-orig for proj, orig in zip (projected, points_2d)])
    resid = np.abs(residuals.flatten()).sum()
    print ("Residuals={}".format(resid))
    machine_resid = sc.single_residual([fx, fy, cx, cy, 0, 0, 0, 0, 0, rx, ry, rz, tx, ty, tz], points_3d, points_2d)
    print ("Machine Residuals={}".format (machine_resid))
    plt.scatter (*zip(*points_2d), color="red")
    plt.scatter (*zip(*projected), color="blue")
    plt.plot([projected[0][0], points_2d[0][0]], [projected[0][1], points_2d[0][1]], color="g", linestyle  ="-")
    plt.show()
    


y=interactive(points_projected,
              fx=FloatSlider(value=6325, min=4000, max=8000, step=1, continuous_update=False),
              fy=FloatSlider(value=6325, min=4000, max=8000, step=1, continuous_update=False), 
              cx=FloatSlider(value=1640, min=500, max=2000, step=1, continuous_update=False),
              cy=FloatSlider(value=1232, min=500, max=2000, step=1, continuous_update=False), 
              rx=FloatSlider(value=-0.11327094, min=-np.pi, max=np.pi, step=0.01, continuous_update=False),
              ry=FloatSlider(value=-0.03599582, min=-np.pi, max=np.pi, step=0.01, continuous_update=False),
              rz=FloatSlider(value=1.55522712, min=-np.pi, max=np.pi, step=0.01, continuous_update=False),
              tx=FloatSlider(value=0.00165974, min=-0.1, max=0.1, step=0.0002, continuous_update=False),
              ty=FloatSlider(value=-0.00299121, min=-0.1, max=0.1, step=0.0002, continuous_update=False),
              tz=FloatSlider(value=0.01946288, min=-0.1, max=0.1, step=0.001, continuous_update=False),
              points_3d=fixed(position), points_2d=fixed(points_2d))
#23.4 mm is the 0 position.

display(y)

interactive(children=(FloatSlider(value=6325.0, continuous_update=False, description='fx', max=8000.0, min=400…

In [5]:
fx = y.children[0].value
fy = y.children[1].value
cx = y.children[2].value
cy = y.children[3].value
rx = y.children[4].value
ry = y.children[5].value
rz = y.children[6].value
tx = y.children[7].value
ty = y.children[8].value
tz = y.children[9].value

#reasonable starting values
camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
distortion = np.array([0, 0, 0, 0, 0])
rvec = np.array([rx, ry, rz])
tvec = np.array([tx, ty, tz])

undistort = sc.calibrate_from_3d_points (position, points_2d, camera_matrix, distortion, rvec, tvec)
print (undistort)

undistort[0].write_file("intrinsics.json")



Actual Fit:
Fit sucessful.
Camera Matrix:
[[6.25560278e+03 0.00000000e+00 1.63300298e+03]
 [0.00000000e+00 6.22861606e+03 1.20690756e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Matrix
[-0.16779224 -0.49556149 -0.00290808  0.00257239 -0.24547894]
rvec
[-0.11356494 -0.02950281  1.5545296 ]
tvec
[ 0.00167885 -0.00294633  0.01930473]
({'__undistort__': True, 'camera_matrix': array([[6.2556025e+03, 0.0000000e+00, 1.6330029e+03],
       [0.0000000e+00, 6.2286162e+03, 1.2069076e+03],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]], dtype=float32), 'distortion': array([-0.16779223, -0.49556148, -0.00290808,  0.00257239, -0.24547894],
      dtype=float32)}, array([[6.25560278e+03, 0.00000000e+00, 1.63300298e+03],
       [0.00000000e+00, 6.22861606e+03, 1.20690756e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), array([-0.16779224, -0.49556149, -0.00290808,  0.00257239, -0.24547894]), array([-0.10159   , -0.03599582,  1.54841   ]), array([ 0.00165974, -